
The Collatz problem is an unsolved problem in mathematics.  It goes like this:

Define the function $f: N \rightarrow N$ on the natural numbers as 
$$
\begin{align}
f(n) =
    \begin{cases}
    \frac{n}{2} & \text{if}\ n \ \text{even} \\
    3n + 1 & \text{otherwise}
    \end{cases}
\end{align}
$$

**Conjecture**: Repeatedly evaluating $f$ on any natural number will always yield 1 after a finite number of steps. [[1]](https://arxiv.org/pdf/math/0309224.pdf)

Note that if $x$ is even, repeated applications of $f$ will divide out all of the zeros.  I thought it might be useful to only looking at the odd numbers visited in this process.

We can work backwards and define the "Collatz Tree" where the $i$'th level is all of the numbers that first yield 1 after $i$ applications of $f$.  Naturally the root is 1 itself.

To compute the tree, we observe that given a node $x$, its children are given by $\frac{2^ix - 1}{3}$ where $2^ix \equiv_3 1$ and $i>0$.  The value of $x$ mod 3 determines the valid values of $i$.  Note that if $x\equiv_30$ then $x$ is a leaf.

Recall that $2^{\text{even}}\equiv_3 1$ and $2^{\text{odd}}\equiv_3 2$, so if $x\equiv_31$ then $i$ even and if $x\equiv_3 2$ then $i$ odd.

This gives the following function to generate children:

- if $x \equiv_3 1$, then children of $x$ are $\frac{2^{2i}x - 1}{3}, i > 0$.
- if $x \equiv_3 2$, then children of $x$ are $\frac{2^{2i+1}x - 1}{3}, i \geq 0$.

We apply this recursively to generate the tree.

I've modified some d3.js code to render the Collatz Tree. [[2]](https://bl.ocks.org/mbostock/4339083) `build_tree` takes in the branching factor, depth and a naming function for each node. Width and height are also parameters to `get_html` as they affect the layout of the tree.

Here is a plot of the tree in base 2 along with its modulo 3 with branching factor 5 and depth 7.

In [1]:
import tree
from IPython.core.display import HTML
tree0 = tree.build_tree(5, 7)
html = tree.get_html(tree0)
HTML(html)

We can see some patterns from the visualization.  I make a slight abuse of notation by adding and multiplying numbers in base 2 and 10 freely. 

**pattern 1**: Looking at the base2 representation, $101X$ takes the same number of steps as $1X$.

**proof**: Let $1Y$ be the parent of $1X$ and note that $101X = 4 \cdot 1X + 1$. 

If $1Y \equiv_3 1$, then $1X = \frac{2^{2i}1Y - 1}{3}$ where $i \geq 1$. 

$\begin{align}
101X &= 2^2 \big(\frac{2^{2i}1Y - 1}{3}\big) + 1 &= 2^2 \big(\frac{2^{2i}1Y - 1}{3}\big) + \frac{3}{3}
&= \frac{2^{2(i+1)}1Y - 4 + 3}{3} &= \frac{2^{2(i+1)}1Y - 1}{3}
\end{align}$

So $1Y$ is the parent of $101X$.

If $1Y \equiv_3 2$, then $1X = \frac{2^{2i+1}1Y - 1}{3}$ where $i \geq 0$ and the proof is analogous.


**pattern 2**: If $1Y \equiv_3 1$, then $101Y \equiv_3 2$ and if first child of $1Y$ is $1X$, then first child of $101Y$ is $11X$. 

**proof**: Note that first child of $1Y$ is $\frac{2^21Y - 1}{3}$ and the first child of $101Y$ is $\frac{2\cdot 101Y -1}{3}$. We see that

$
\begin{align}
\frac{2\cdot 101Y -1}{3} = \frac{2 (2^2 1Y + 1) - 1}{3} = \frac{2 \cdot 2^21Y + 2 - 1}{3} = \frac{2 \cdot 2^21Y + 1}{3} = \frac{2 \cdot 2^21Y - 2 + 1 + 2}{3} = \frac{2\cdot(2^21Y - 1) + 3}{3} = 2\big[\frac{2^21Y - 1}{3}\big] + 1
\end{align}
$

This means that the first child of $101Y$ is $2\cdot 1X + 1 = 11X$.

**pattern 3**: 
- If $Y \equiv_3 1$ and first child of $1Y$ is $1X$, then first child of $1010101Y$ is $1000111X$.
- If $Y \equiv_3 2$ and first child of $1Y$ is $11X$, then first child of $1010101Y$ is $11000111X$.

**proof**: Note that $1X = \frac{2^21Y - 1}{3}$, $1010101Y = 2^0 + 2^2 + 2^4 + 2^61Y$ and $1000111X = 2^0 + 2^4 + 2^5 + 2^61X$.

We see that
$
\begin{align}
\frac{2^21010101Y - 1}{3} &= \frac{2^2[2^61Y + 2^0 + 2^2 + 2^4] - 1}{3} = \frac{2^22^61Y + 2^2 + 2^4 + 2^6 - 1}{3} = \frac{2^22^61Y + 3 + 2^4 + 2^6}{3}
\end{align}
$

Similarly
$
1000111X = 2^0 + 2^4 + 2^5 + 2^61X = 2^0 + 2^4 + 2^5 + 2^6\frac{2^21Y - 1}{3}
= \frac{3\cdot[2^0 + 2^4 + 2^5] + 2^22^61Y - 2^6}{3} = \frac{2^0 + 2^4 + 2^5 + 2^1 + 2^5 + 2^6 + 2^22^61Y - 2^6}{3} = \frac{2^22^61Y + 3 + 2^4 + 2^6}{3}
$

The second point follows from pattern 2.



**pattern 4**: Numbers of the form $1\cdots101 \equiv_3 2$

**proof**: Simple induction.  Base case is 101.  Inductive step: Assume true for $X=1\cdots101$ with $k$ ones. Then $k+1$ ones is equivalent to $2\cdot X + 1 \equiv_3 2\cdot2 + 1 \equiv_3 2$.




I tried doing the same thing with the tree plotted in base 3.  I only found 1 pattern.

In [2]:
import collatz
tree1 = tree.build_tree(5, 7, collatz.base3)
html = tree.get_html(tree1)
HTML(html)

**pattern 5**: If $1Y \equiv_3 1$ and $101Y$ is $2X$ in base 3, then first child of $1Y$ is $X$ in base 3.

**proof**: First child of $1Y$ is $\frac{2^21Y - 1}{3}$. We see that
$
\begin{align}
2X = 3\cdot X + 2 = 3 \frac{2^21Y - 1}{3} + 2 = 2^21Y - 1 + 2 = 2^21Y + 2 = 101Y 
\end{align}
$

# Discussion

We can see a lot of regularities in the visualizations, particularly in base 2, which makes me believe that the Collatz conjecture is true or not provable.  I'd be pretty surprised if there's a class of numbers that's not enumerated by it.

I was hoping that exhaustively applying the patterns would reduce to something useful.  One might imagine that they reduce a node to the first node at that level or to 1. For example, $1010100011101 \rightarrow 100011101 \rightarrow 1101$ via repeated applications of pattern 1 then an application of pattern 3.  Then it would be a matter of showing that whatever you end up with is something that reduces to 1, in this case 1101 is part of the sequence of first nodes at levels in the tree.  

As I started to write this out, however, I noticed some problems

```
Case 101:
    apply pattern 1.

Case 100:
    if first 7 bits are 1000111: 
        apply pattern 3.
    else: 
        case 1000000:
        case 1000001:
        case 1000010:
        case 1000100:
        case 1001000:
        ...

Case 111:
    if parent % 3 == 2:
        if parent starts with 101:
            apply pattern 2.
        else:
            don't know.
    else:
        apply pattern 5.
 
Case 110:
    if parent % 3 == 2:
      if parent starts with 101:
          apply pattern 2.
      else:
          don't know.
    else:
      apply pattern 5.

```

First, the patterns didn't come close to covering all the cases.  There was always a path for some numbers to wiggle out.  Next, the patterns seemed to depend on the depth of the first common ancestor.  I only found ones up to common grand parents because they're easier spot. I suspect there are more patterns for each level as we consider further ancestors (probably involving longer base 2 representations) and that there are an infinite number of cases to consider as a result.  Finally, the proofs of the patterns were all pretty elementary and pretty much the same, so that could be a signal that this isn't going to work.

If you have to consider an infinite number of cases, that doesn't seem any better than just running the Collatz function itself.  Any arguments for progress will also get stumped if you don't cover all of them because there will be something left over after you've exhaustively applied all your patterns and you can't say anything about it.  

It would be interesting if we can't prove the Collatz conjecture is true or not.  The halting problem says that there isn't a general algorithm that determines if an arbitrary program will halt on an arbitrary input. [[3]](http://www-math.mit.edu/~poonen/papers/h10_notices.pdf)  In this case, we're fixing the program and asking if it will terminate on an arbitrary input.  

Still, I think these visualizations (base 2 in particular) reveal more structure than how the execution tree is usually depicted. [[4]](https://en.wikipedia.org/wiki/Collatz_conjecture#Visualizations)  Hopefully they can be of use to someone who's working on the problem.  See [here](https://github.com/dkamm/collatz) for the code I used.
